# <font color=#0c2290> Projet détection de fraudes au chronotackygraphe 📊

def generer_rapport(df_suspects, fichier_sortie='rapport_fraudes.csv'):
    """
    Génère un rapport des cas suspects
    """
    if len(df_suspects) == 0:
        print("\n✓ Aucune fraude potentielle détectée")
        return
    
    # Colonnes à inclure dans le rapport
    colonnes_rapport = [
        'nom',
        'adresse',
        'activite',
        'distance_domicile_km',
        'score_risque',
        'latitude_domicile',
        'longitude_domicile',
        'latitude_position',
        'longitude_position'
    ]
    
    # Filtrer les colonnes existantes
    colonnes_disponibles = [col for col in colonnes_rapport if col in df_suspects.columns]
    df_rapport = df_suspects[colonnes_disponibles].copy()
    
    # Arrondir les valeurs numériques
    df_rapport['distance_domicile_km'] = df_rapport['distance_domicile_km'].round(2)
    df_rapport['score_risque'] = df_rapport['score_risque'].round(1)
    
    # Sauvegarder le rapport
    df_rapport.to_csv(fichier_sortie, index=False, encoding='utf-8-sig')
    
    print(f"\n {len(df_suspects)} cas suspects détectés!")
    print(f"\nTop 5 des cas les plus suspects:")
    print("-" * 80)
    
    for idx, row in df_rapport.head(5).iterrows():
        print(f"\nChauffeur: {row['nom']}")
        print(f"  Distance du domicile: {row['distance_domicile_km']:.2f} km")
        print(f"  Score de risque: {row['score_risque']:.1f}/100")
        print(f"  Activité: {row.get('activite', 'N/A')}")
    
    print(f"\n✓ Rapport complet sauvegardé dans: {fichier_sortie}")


def main():
    """
    Fonction principale
    """
    print("=" * 80)
    print("DÉTECTION DE FRAUDES POTENTIELLES - CHAUFFEURS")
    print("=" * 80)
    print()
    
    # Configuration
    fichier_domiciles = 'chauffeurs_domiciles.csv'
    fichier_positions = 'chauffeurs_positions.csv'
    seuil_distance = 5  # km
    
    print(f"Paramètres:")
    print(f"  - Seuil de distance: {seuil_distance} km")
    print()
    
    # Charger les données
    df_domiciles = charger_donnees_domiciles(fichier_domiciles)
    df_positions = charger_donnees_positions(fichier_positions)
    
    if df_domiciles is None or df_positions is None:
        print("\n Impossible de charger les données")
        return
    
    # Détecter les fraudes
    print("\nAnalyse en cours...")
    df_suspects = detecter_fraudes(df_domiciles, df_positions, seuil_distance)
    
    # Générer le rapport
    generer_rapport(df_suspects)
    
    print("\n" + "=" * 80)
    print("Analyse terminée")
    print("=" * 80)


## **Importer les bibliothèques**

import pandas as pd
from math import radians, cos, sin, asin, sqrt
from datetime import datetime

### **Calcul des kilomètres entre deux points (Latitude / Longitude)**

def calculer_distance_haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Calcule la distance en kilomètres entre deux points
    géographiques en utilisant la formule de Haversine.

    Parameters
    ----------
    lat1 : float -> Latitude du point de départ (en degrés).
    lon1 : float -> Longitude du point de départ (en degrés).
    lat2 : float -> Latitude du point d'arrivée (en degrés).
    lon2 : float -> Longitude du point d'arrivée (en degrés).

    Returns
    -------
    float -> Distance en kilomètres entre les deux points.
    """

    # Conversion des degrés en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calcul des différences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Application de la formule de Haversine
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))

    # Rayon de la Terre en kilomètres
    r = 6371

    return c * r

### Exemple d'utilisation

if __name__ == "__main__":
    # Paris -> Lyon
    distance = calculer_distance_haversine(48.8566, 2.3522, 45.7640, 4.8357)
    print(f"Distance Paris -> Lyon : {distance:.2f} km")

## **Chargement du fichier des positions des domiciles des chauffeurs**
## ===============================================================

### **Chemin vers le fichier CSV**
df = pd.read_csv(
    "Users", "bourtguize", "Desktop", "dataSuits",
    "FORMATION DATA ANALYST - Septembre 2025",
    "DOSSIER MODULE 6 - Projet final & Prépa pro",
    "Data-projet", "data", "personnel_coordonnees.csv"
)

### **Liste des encodages à tester**
ENCODAGES = ["utf-8", "latin-1", "cp1252", "iso-8859-1"]

### **Colonnes requises après renommage**
COLONNES_REQUISES = ["nom", "latitude_domicile", "longitude_domicile"]

#### **=============================================================**
#### Format attendu : nom, latitude_domicile, longitude_domicile
#### **=============================================================**

### **Correspondance des colonnes brutes -> nom attendu**
COLONNES_RENOMMAGE = {
    "Nom":       "nom",
    "latitude":  "latitude_domicile",
    "longitude": "longitude_domicile",
}


def load_domiciles_file(file_path: str = FILE_PATH) -> pd.DataFrame | None:
    """
    Charge le fichier CSV des domiciles des chauffeurs.

    Tente plusieurs encodages successivement jusqu'à un
    chargement réussi. Vérifie ensuite la présence des
    colonnes obligatoires après renommage.

    Parameters
    ----------
    file_path : str -> Chemin vers le fichier CSV.

    Returns
    -------
    pd.DataFrame | None
        DataFrame nettoyé si le fichier est valide,
        None sinon.
    """

    for encoding in ENCODAGES:
        try:
            # --- Lecture du fichier CSV ---
            df = pd.read_csv(file_path, encoding=encoding, sep=";")

            # --- Renommage des colonnes ---
            df = df.rename(columns=COLONNES_RENOMMAGE)

            # --- Vérification des colonnes requises ---
            colonnes_manquantes = [
                col for col in COLONNES_REQUISES if col not in df.columns
            ]

            if colonnes_manquantes:
                print(f"✗ Colonnes manquantes : {colonnes_manquantes}")
                print(f"  Colonnes présentes  : {list(df.columns)}")
                return None

            # --- Retour du DataFrame validé ---
            print(f"✓ {len(df)} chauffeurs chargés (encodage : {encoding})")
            return df

        except (UnicodeDecodeError, UnicodeError):
            # Encodage incompatible -> on teste le suivant
            continue

        except Exception as e:
            print(f"✗ Erreur avec l'encodage '{encoding}' : {e}")
            continue

    # Aucun encodage ne a fonctionné
    print("✗ Aucun encodage compatible trouvé pour le fichier.")
    return None

#### Test : uniquement si ouvre fichier directement
### =============================================================
if __name__ == "__main__":

    # Chargement du fichier domiciles
    df_domiciles = load_domiciles_file()

    if df_domiciles is not None:
        print("\n--- Aperçu des données ---")
        print(df_domiciles.head())
    else:
        print("\n✗ Échec du chargement du fichier.")

### **def charger_donnees_positions(fichier_positions):**
## ======================================================

    encodages = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252', 'utf-16']
    
    for encoding in encodages:
        try:
            # Ajouter sep=';' pour le séparateur point-virgule
            df = pd.read_csv("/Users/bourtguize/Desktop/dataSuits/FORMATION DATA ANALYST - Septembre 2025/DOSSIER MODULE 6 - Projet final & Prépa pro/Data-projet/temps de service_juin.csv", encoding=encoding, sep=';')
            
            # Renommer les colonnes pour correspondre aux noms attendus
            df = df.rename(columns={
                'Chauffeur': 'nom',
                'Latitude': 'latitude_position',
                'Longitude': 'longitude_position',
                'Code de travail': 'activite'
            })
            
            # Vérifier les colonnes requises
            colonnes_requises = ['nom', 'latitude_position', 'longitude_position', 'activite']
            colonnes_manquantes = [col for col in colonnes_requises if col not in df.columns]
            
            if colonnes_manquantes:
                print(f"Colonnes manquantes dans le fichier positions: {colonnes_manquantes}")
                print(f"Colonnes présentes: {list(df.columns)}")
                return None
            
            print(f"✓ {len(df)} positions chargées depuis le fichier positions (encodage: {encoding})")
            print(f"  Colonnes: {list(df.columns)}")
            return df
            
        except (UnicodeDecodeError, UnicodeError):
            continue
        except Exception as e:
            print(f"Erreur avec encodage {encoding}: {e}")
            continue
    
    print(f"Impossible de lire le fichier avec les encodages testés: {encodages}")
    return None

In [681]:
df_test2 = pd.read_excel("/Users/bourtguize/Desktop/dataSuits/FORMATION DATA ANALYST - Septembre 2025/DOSSIER MODULE 6 - Projet final & Prépa pro/Data-projet/temps de service_juin.xlsx")
print("Colonnes:", df_test2.columns.tolist())
print()
print(df_test2.head())

Colonnes: ['Date et heure de début', 'Date et heure de fin', 'Longitude', 'Latitude', 'Position', 'Véhicule', 'Chauffeur', 'Code de travail', 'Description', 'Durée']

  Date et heure de début Date et heure de fin  Longitude  Latitude  \
0    2024-06-07 07:01:00  2024-06-07 07:03:00     4.0002   43.6516   
1    2024-06-07 07:03:00  2024-06-07 07:07:00     4.0002   43.6516   
2    2024-06-07 07:07:00  2024-06-07 07:28:00     3.9994   43.6587   
3    2024-06-07 07:28:00  2024-06-07 07:40:00     3.9992   43.6587   
4    2024-06-07 07:40:00  2024-06-07 08:47:00     3.9359   43.5935   

                                            Position           Véhicule  \
0  FRA - Rue Jules Milhau (1-) - 34670 - Baillarg...  K1619 (FR-898-VC)   
1  FRA - Rue Jules Milhau (1-) - 34670 - Baillarg...  K1619 (FR-898-VC)   
2  FRA - Avenue de la Biste (350-) - 34670 - Bail...  K1619 (FR-898-VC)   
3  FRA - Avenue de la Biste (350-) - 34670 - Bail...  K1619 (FR-898-VC)   
4  FRA - Route de Vauguières (-) - 34

In [682]:
# À exécuter AVANT d'appeler detecter_fraudes
df_positions = df_positions.rename(columns={
    'Chauffeur': 'nom',
    'Latitude': 'latitude_position',
    'Longitude': 'longitude_position',
    'Code de travail': 'activite'
})
print("✓ Colonnes renommées")

✓ Colonnes renommées


In [683]:
# Configuration
fichier_domiciles = '/Users/bourtguize/Desktop/dataSuits/FORMATION DATA ANALYST - Septembre 2025/DOSSIER MODULE 6 - Projet final & Prépa pro/Data-projet/data/liste_personnel_avec_coordonnees_1.csv'

fichier_positions = '/Users/bourtguize/Desktop/dataSuits/FORMATION DATA ANALYST - Septembre 2025/DOSSIER MODULE 6 - Projet final & Prépa pro/Data-projet/temps de service_juin.csv'

seuil_distance = 0.5  # km

In [684]:
def process_data(df_merged, seuil_distance_km= 0.5):
    # Fix the first line that had a syntax error
    df_merged['Date et heure de début'] = pd.to_datetime(df_merged['Date et heure de début'])
    df_merged['Date et heure de fin'] = pd.to_datetime(df_merged['Date et heure de fin'])
    df_merged['Jour'] = df_merged['Date et heure de début'].dt.strftime('%Y-%m-%d')
    
    # Convertir datetime.time en minutes
    def time_to_minutes(time_obj):
        if pd.isna(time_obj):
            return 0
        try:
            # Si c'est un objet time
            if hasattr(time_obj, 'hour'):
                return time_obj.hour * 60 + time_obj.minute + time_obj.second / 60
            # Si c'est une chaîne
            else:
                parts = str(time_obj).split(':')
                return int(parts[0]) * 60 + int(parts[1]) + (int(parts[2]) if len(parts) > 2 else 0) / 60
        except:
            return 0
    
    df_merged['Durée_minutes'] = df_merged['Durée'].apply(time_to_minutes)
    
    # Filtrer uniquement les chauffeurs "en travail"
    df_travail = df_merged[df_merged['Description'].str.lower().str.contains('travail', na=False)]
    
    if len(df_travail) == 0:
        print("Aucun chauffeur en activité de travail trouvé")
        return pd.DataFrame()
    
    # Calculer la distance pour chaque position
    df_travail = df_travail.copy()
    df_travail['distance_domicile_km'] = df_travail.apply(
        lambda row: calculer_distance_haversine(
            row['latitude_domicile'],
            row['longitude_domicile'],
            row['latitude_position'],
            row['longitude_position']
        ),
        axis=1
    )
    
    # Identifier les cas suspects (distance < seuil)
    df_suspects = df_travail[df_travail['distance_domicile_km'] < seuil_distance_km].copy()
    
    # Calculer un score de risque (plus proche = plus suspect)
    df_suspects['score_risque'] = (1 - (df_suspects['distance_domicile_km'] / seuil_distance_km)) * 100
    
    # Trier par score de risque décroissant
    df_suspects = df_suspects.sort_values('score_risque', ascending=False)
    
    return df_suspects

In [685]:
print(type(df_positions['Durée'].iloc[0]))
print(df_positions['Durée'].head())

<class 'datetime.time'>
0    00:14:00
1    00:25:00
2    00:17:00
3    00:01:00
4    00:05:00
Name: Durée, dtype: object


In [686]:
print("\n=== DÉTECTION DES FRAUDES ===")
df_suspects = detecter_fraudes(df_domiciles, df_positions, seuil_distance)

if df_suspects is not None and len(df_suspects) > 0:
    print(f"\n {len(df_suspects)} cas suspects détectés !")
    print("\nAperçu des cas suspects :")
    print(df_suspects[['nom', 'distance_domicile_km', 'score_risque']].head(10))
    
    # Saugegarder
    chemin_sortie = '/Users/bourtguize/Desktop/fraudes_detectees.csv'
    df_suspects.to_csv(chemin_sortie, index=False, sep=';', encoding='utf-8')
    print(f"\n✓ Fichier sauvegardé sur le Bureau : fraudes_detectees.csv")
else:
    print("\nAucune fraude détectée")


=== DÉTECTION DES FRAUDES ===
Colonnes dans df_merged après fusion:
['Date et heure de début', 'Position', 'latitude_position', 'longitude_position', 'Date et heure de fin', 'Véhicule', 'nom', 'activite', 'Description', 'Durée', 'Prénom', "Date d'embauche", 'Date de naissance', 'Carte N°', 'Adresse 1', 'code_postal', 'commune', 'latitude_domicile', 'longitude_domicile']

Aucun chauffeur en activité de travail trouvé

Aucune fraude détectée


In [687]:
def main():
    
    print("=" * 80)
    print("DÉTECTION DE FRAUDES POTENTIELLES - CHAUFFEURS")
    print("=" * 80)
    print()

In [688]:
# Configuration
fichier_domiciles = '/Users/bourtguize/Desktop/dataSuits/FORMATION DATA ANALYST - Septembre 2025/DOSSIER MODULE 6 - Projet final & Prépa pro/Data-projet/data/liste_personnel_avec_coordonnees_1.csv'

fichier_positions = '/Users/bourtguize/Desktop/dataSuits/FORMATION DATA ANALYST - Septembre 2025/DOSSIER MODULE 6 - Projet final & Prépa pro/Data-projet/temps de service_juin.csv'

seuil_distance = 0,5 # km

print(f"Paramètres:")
print(f"  - Seuil de distance: {seuil_distance} km")
print()

Paramètres:
  - Seuil de distance: (0, 5) km



## **Détecter les fraudes**
## ==============================
print("\nAnalyse en cours...")

# Renommer les colonnes si nécessaire
if 'Chauffeur' in df_positions.columns and 'nom' not in df_positions.columns:
    # Check if 'Code de travail' exists before renaming
    rename_dict = {
        'Chauffeur': 'nom',
        'Latitude': 'latitude_position',
        'Longitude': 'longitude_position'
    }
    
    # Only add 'Code de travail' to rename dict if it exists
    if 'Code de travail' in df_positions.columns:
        rename_dict['Code de travail'] = 'activite'
    
    df_positions = df_positions.rename(columns=rename_dict)
    print("✓ Colonnes de df_positions renommées")

# Supprimer les colonnes en double
df_positions = df_positions.loc[:, ~df_positions.columns.duplicated()]

print("✓ Colonnes en double supprimées")
print("Colonnes finales:", df_positions.columns.tolist())

# Check if df_domiciles and df_positions are not None before proceeding
if df_domiciles is None:
    print("Erreur: df_domiciles est None. Veuillez vérifier les étapes précédentes.")
    # Initialize with empty DataFrame if needed
    # df_domiciles = pd.DataFrame()
elif df_positions is None:
    print("Erreur: df_positions est None. Veuillez vérifier les étapes précédentes.")
    # Initialize with empty DataFrame if needed
    # df_positions = pd.DataFrame()
else:
    print("\nAnalyse en cours...")
    # Make sure the detecter_fraudes function doesn't require 'activite' column
    # or handle its absence properly
    df_suspects = detecter_fraudes(df_domiciles, df_positions, seuil_distance)
    
    # Check if df_suspects is not None before generating report
    if df_suspects is not None:
        # Générer le rapport
        generer_rapport(df_suspects)
    else:
        print("Erreur: La détection de fraudes n'a pas retourné de résultats valides.")

In [690]:
import pandas as pd

fichier_csv = '/Users/bourtguize/Desktop/dataSuits/FORMATION DATA ANALYST - Septembre 2025/DOSSIER MODULE 6 - Projet final & Prépa pro/Data-projet/data/liste_personnel_avec_coordonnees_1.csv'

# Essayer de charger avec différents séparateurs
try:
    # Essai 1 : avec point-virgule
    df_test = pd.read_csv(fichier_csv, sep=';', encoding='utf-8')
    print("✓ Chargement réussi avec sep=';'")
    print(f"Lignes : {len(df_test)}")
    print(f"Colonnes : {df_test.columns.tolist()}")
except Exception as e:
    print(f"Erreur avec sep=';' : {e}")
    
    try:
        # Essai 2 : avec virgule
        df_test = pd.read_csv(fichier_csv, sep=',', encoding='utf-8')
        print("✓ Chargement réussi avec sep=','")
        print(f"Lignes : {len(df_test)}")
        print(f"Colonnes : {df_test.columns.tolist()}")
    except Exception as e2:
        print(f"Erreur avec sep=',' : {e2}")

✓ Chargement réussi avec sep=';'
Lignes : 105
Colonnes : ['Nom', 'Prénom', "Date d'embauche", 'Date de naissance', 'Carte N°', 'Adresse 1', 'code_postal', 'commune', 'latitude', 'longitude']


In [691]:
print("=== VÉRIFICATION DES DONNÉES ===")
print(f"df_domiciles existe : {'df_domiciles' in locals()}")
print(f"df_positions existe : {'df_positions' in locals()}")

if 'df_domiciles' in locals() and df_domiciles is not None:
    print(f"✓ df_domiciles : {len(df_domiciles)} lignes")
    print(f"  Colonnes : {df_domiciles.columns.tolist()}")
else:
    print("✗ df_domiciles n'est pas chargé correctement")

if 'df_positions' in locals() and df_positions is not None:
    print(f"✓ df_positions : {len(df_positions)} lignes")
    print(f"  Colonnes : {df_positions.columns.tolist()}")
else:
    print("✗ df_positions n'est pas chargé correctement")

=== VÉRIFICATION DES DONNÉES ===
df_domiciles existe : True
df_positions existe : True
✓ df_domiciles : 105 lignes
  Colonnes : ['nom', 'Prénom', "Date d'embauche", 'Date de naissance', 'Carte N°', 'Adresse 1', 'code_postal', 'commune', 'latitude_domicile', 'longitude_domicile']
✓ df_positions : 37411 lignes
  Colonnes : ['Date et heure de début', 'Position', 'latitude_position', 'longitude_position', 'Date et heure de fin', 'Véhicule', 'nom', 'activite', 'Description', 'Durée']


In [692]:
# Recharger df_domiciles
print("=== RECHARGEMENT DE df_domiciles ===")
df_domiciles = charger_donnees_domiciles(fichier_domiciles)

if df_domiciles is not None:
    print(f"✓ df_domiciles chargé : {len(df_domiciles)} lignes")
    print(f"  Colonnes : {df_domiciles.columns.tolist()}")
    
    # Vérifier les colonnes nécessaires
    print("\n=== VÉRIFICATION DES COLONNES ===")
    print(f"'nom' présent : {'nom' in df_domiciles.columns}")
    print(f"'latitude_domicile' présent : {'latitude_domicile' in df_domiciles.columns}")
    print(f"'longitude_domicile' présent : {'longitude_domicile' in df_domiciles.columns}")
else:
    print("ERREUR : df_domiciles n'a pas pu être chargé")
    print("\nEssayons de charger manuellement...")
    
    # Chargement manuel
    df_domiciles = pd.read_csv(fichier_domiciles, sep=';', encoding='utf-8')
    df_domiciles = df_domiciles.rename(columns={
        'Nom': 'nom',
        'latitude': 'latitude_domicile',
        'longitude': 'longitude_domicile'
    })
    print(f"✓ Chargement manuel réussi : {len(df_domiciles)} lignes")
    print(f"  Colonnes : {df_domiciles.columns.tolist()}")

=== RECHARGEMENT DE df_domiciles ===
Colonnes manquantes dans le fichier domiciles: ['latitude_domicile', 'longitude_domicile']
Colonnes présentes: ['Début Edition', 'Fin Edition', 'nom', 'Prénom', 'Véhicule', 'Kilomètres', 'Amplitude', 'Effectif', 'Rémunéré', 'Jours travaillés', "Jours d'absence", 'Travail', 'Conduite', 'Repos']
ERREUR : df_domiciles n'a pas pu être chargé

Essayons de charger manuellement...
✓ Chargement manuel réussi : 105 lignes
  Colonnes : ['nom', 'Prénom', "Date d'embauche", 'Date de naissance', 'Carte N°', 'Adresse 1', 'code_postal', 'commune', 'latitude_domicile', 'longitude_domicile']


In [693]:
# Lancer la détection
print("\n=== DÉTECTION DES FRAUDES ===")
df_suspects = detecter_fraudes(df_domiciles, df_positions, seuil_distance)

if df_suspects is not None and len(df_suspects) > 0:
    print(f"\n✓ {len(df_suspects)} cas suspects détectés !")
    
    # Afficher un aperçu
    print("\nAperçu des cas suspects :")
    print(df_suspects[['nom', 'distance_domicile_km', 'score_risque', 'Durée_minutes']].head(10))
    
    # Sauvegarder
    chemin_sortie = '/Users/bourtguize/Desktop/fraudes_detectees.csv'
    df_suspects.to_csv(chemin_sortie, index=False, sep=';', encoding='utf-8')
    print(f"\n✓ Fichier sauvegardé sur le Bureau : fraudes_detectees.csv")
else:
    print("\nAucune fraude détectée avec les critères actuels")
    print(f"Seuil de distance utilisé : {seuil_distance} km")


=== DÉTECTION DES FRAUDES ===
Colonnes dans df_merged après fusion:
['Date et heure de début', 'Position', 'latitude_position', 'longitude_position', 'Date et heure de fin', 'Véhicule', 'nom', 'activite', 'Description', 'Durée', 'Prénom', "Date d'embauche", 'Date de naissance', 'Carte N°', 'Adresse 1', 'code_postal', 'commune', 'latitude_domicile', 'longitude_domicile']

Aucun chauffeur en activité de travail trouvé

Aucune fraude détectée avec les critères actuels
Seuil de distance utilisé : (0, 5) km


In [694]:
# Voir les valeurs dans 'activite' et 'Description'
print("=== Valeurs uniques dans 'activite' ===")
print(df_positions['activite'].value_counts())
print()
print("=== Valeurs uniques dans 'Description' ===")
print(df_positions['Description'].value_counts())

=== Valeurs uniques dans 'activite' ===
activite
AA    17595
RD    16631
AR     3134
AO       51
Name: count, dtype: int64

=== Valeurs uniques dans 'Description' ===
Description
Travail     17595
Conduite    16631
Repos        3134
Dispo          51
Name: count, dtype: int64


In [695]:
# Fusion manuelle pour vérifier
df_merged_test = pd.merge(df_positions, df_domiciles, on='nom', how='inner')

print(f"=== RÉSULTAT DE LA FUSION ===")
print(f"Lignes après fusion : {len(df_merged_test)}")
print()

# Vérifier les noms dans les deux DataFrames
print("=== EXEMPLES DE NOMS dans df_positions ===")
print(df_positions['nom'].unique()[:10])
print()
print("=== EXEMPLES DE NOMS dans df_domiciles ===")
print(df_domiciles['nom'].unique()[:10])
print()

# Vérifier la colonne Description après fusion
if len(df_merged_test) > 0:
    print("=== Valeurs dans 'Description' après fusion ===")
    print(df_merged_test['Description'].value_counts())
    print()
    
    # Filtrer sur Travail
    df_travail = df_merged_test[df_merged_test['Description'].str.lower().str.contains('travail', na=False)]
    print(f"Lignes avec 'Travail' : {len(df_travail)}")
    
    if len(df_travail) > 0:
        print("\n=== EXEMPLE DE DONNÉES ===")
        print(df_travail[['nom', 'latitude_position', 'longitude_position', 'latitude_domicile', 'longitude_domicile', 'Description']].head())
else:
    print("⚠ AUCUNE CORRESPONDANCE lors de la fusion !")
    print("Les noms ne correspondent pas entre les deux fichiers")

=== RÉSULTAT DE LA FUSION ===
Lignes après fusion : 37261

=== EXEMPLES DE NOMS dans df_positions ===
['ODIN' 'PRUDHOMME' 'RICHET' 'LAGRAOUI' 'POIRIER-DIT-CAULIER' 'ABERT'
 'BIARGE' 'BENOIT' 'BOISSIERE' 'MARTY']

=== EXEMPLES DE NOMS dans df_domiciles ===
['BENOIT' 'LE GAGNE' 'TAILLASSON' 'BUMENN' 'FERREIRA NETO' 'PRUDHOMME'
 'CARVALHO PINTO' 'LANSARD' 'GREITER' 'MALCHEAUX']

=== Valeurs dans 'Description' après fusion ===
Description
Travail     17534
Conduite    16574
Repos        3098
Dispo          55
Name: count, dtype: int64

Lignes avec 'Travail' : 17534

=== EXEMPLE DE DONNÉES ===
    nom  latitude_position  longitude_position  latitude_domicile  \
0  ODIN            43.6516              4.0003            43.6889   
2  ODIN            43.7402              4.2218            43.6889   
4  ODIN            43.7405              4.2218            43.6889   
6  ODIN            43.7363              4.2335            43.6889   
8  ODIN            43.6581              3.9988            4

In [678]:
# Extraire le dernier mot du nom (nom de famille) dans df_positions
df_positions['nom'] = df_positions['nom'].str.strip().str.split(' ').str[-1].str.upper()

print("=== VÉRIFICATION APRÈS CORRECTION ===")
print("Exemples de noms dans df_positions :")
print(df_positions['nom'].unique()[:10])
print()
print("Exemples de noms dans df_domiciles :")
print(df_domiciles['nom'].unique()[:10])

=== VÉRIFICATION APRÈS CORRECTION ===
Exemples de noms dans df_positions :
['ODIN' 'PRUDHOMME' 'RICHET' 'LAGRAOUI' 'POIRIER-DIT-CAULIER' 'ABERT'
 'BIARGE' 'BENOIT' 'BOISSIERE' 'MARTY']

Exemples de noms dans df_domiciles :
['BENOIT' 'LE GAGNE' 'TAILLASSON' 'BUMENN' 'FERREIRA NETO' 'PRUDHOMME'
 'CARVALHO PINTO' 'LANSARD' 'GREITER' 'MALCHEAUX']


In [679]:
# Lancer la détection
print("\n=== DÉTECTION DES FRAUDES ===")
df_suspects = detecter_fraudes(df_domiciles, df_positions, seuil_distance)

if df_suspects is not None and len(df_suspects) > 0:
    print(f"\n✓ {len(df_suspects)} cas suspects détectés !")
    print("\nAperçu des cas suspects :")
    print(df_suspects[['nom', 'distance_domicile_km', 'score_risque']].head(10))
    
    # Sauvegarder
    chemin_sortie = '/Users/bourtguize/Desktop/fraudes_detectees.csv'
    df_suspects.to_csv(chemin_sortie, index=False, sep=';', encoding='utf-8')
    print(f"\n✓ Fichier sauvegardé sur le Bureau : fraudes_detectees.csv")
else:
    print("\nAucune fraude détectée")


=== DÉTECTION DES FRAUDES ===
Colonnes dans df_merged après fusion:
['Date et heure de début', 'Position', 'latitude_position', 'longitude_position', 'Date et heure de fin', 'Véhicule', 'nom', 'activite', 'Description', 'Durée', 'Prénom', "Date d'embauche", 'Date de naissance', 'Carte N°', 'Adresse 1', 'code_postal', 'commune', 'latitude_domicile', 'longitude_domicile']

Aucun chauffeur en activité de travail trouvé

Aucune fraude détectée
